In [1]:
from sympy import symbols, pi, log, cos, I, Abs, sqrt, tanh, Integral, lambdify
from sympy.solvers import solve
import numpy as np
from scipy.integrate import dblquad
from scipy.optimize import minimize

from sympy import init_printing
from IPython.display import Markdown
init_printing()

In [2]:
kB = 8.617333e-2 # meV/K

In [3]:
J, J_2, J_3, J_c, J_2c = symbols(r'J J_2 J_3 J_c J_2c')
Gamma, Gamma_c, Gamma_2c, D, D_c = symbols(r'\Gamma \Gamma_c \Gamma_2c D D_c')
k_x, k_y = symbols(r'k_x k_y')
beta = symbols(r'\beta')
T_N, pm = symbols(r'T_N pm')

In [4]:
gamma_k = (cos(k_x) + cos(k_y))/2
gamma_2k = (cos(2*k_x) + cos(2*k_y))/2

B_k = ((8*J + 8*Gamma + J_c + Gamma_c)/2 - 2*J_2*(1 - cos(k_x)*cos(k_y))
       -4*J_3*(1-gamma_2k) - 4*J_2c*(1-pm*gamma_k)
      )
C_k = (8*J*gamma_k + pm*J_c)/2 - I*(8*D*gamma_k + pm*D_c)/2

E = sqrt(B_k**2 - Abs(C_k)**2)

Check the magnon energies at high symmetry points

In [5]:
params = [
    (J, 93./2),
    (J_2, 11.9/2),
    (J_3, 14.6/2),
    (J_2c, 6.2),
    (Gamma, 4.4/2),
    (J_c, 25.2),
    (Gamma_c, 34.3),
    (D, 24.5/2),
    (D_c, 28.1)]

E_subs = E.subs(params)
E_p = E.subs(pm, 1)
E_m = E.subs(pm, -1)

In [6]:
E_pi0 = E_subs.subs([(k_x, pi), (k_y, 0), (pm, 1)])
E_pipi = E_subs.subs([(k_x, pi), (k_y, pi), (pm, -1)])

Markdown(f"Magnon energies at ($\pi$, 0) = {E_pi0:.1f} meV and ($\pi$, $\pi$)={E_pipi:.1f} meV")

Magnon energies at ($\pi$, 0) = 174.9 meV and ($\pi$, $\pi$)=83.7 meV

In [7]:
integrand = (B_k.subs(pm, 1)/(E_p*tanh(beta*E_p)) + B_k.subs(pm, -1)/(E_m*tanh(beta*E_m)) )
S = 1 - (1/(16*pi**2)) *  Integral(integrand, (k_x, -pi, pi), (k_y, -pi, pi))

In [8]:
beta_val = 1. /(kB*285)
I = lambdify([k_x, k_y], integrand.subs(params + [(beta, beta_val)]))
total, error = dblquad(I, -np.pi, np.pi, lambda x: -np.pi, lambda x: np.pi, epsrel=1e-2)

/Users/markdean/anaconda3/envs/main/lib/python3.7/site-packages/scipy/integrate/quadpack.py:448: ComplexWarning: Casting complex values to real discards the imaginary part
  return _quadpack._qagse(func,a,b,args,full_output,epsabs,epsrel,limit)
/Users/markdean/anaconda3/envs/main/lib/python3.7/site-packages/scipy/integrate/quadpack.py:860: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  **opt)
/Users/markdean/anaconda3/envs/main/lib/python3.7/site-packages/scipy/integrate/quadpack.py:860: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  **opt)


In [ ]:
def get_S(T):
    print(f"T={T}")
    beta_val = 1. /(kB*T)
    I = lambdify([k_x, k_y], integrand.subs(params + [(beta, beta_val)]))
    total, error = dblquad(I, -np.pi, np.pi, lambda x: -np.pi, lambda x: np.pi, epsrel=1e-2)
    S = 1 - total/(16*np.pi**2)
    print(f"S={S}")
    return np.abs(S)**2


minimize(get_S, 285)

T=[285.]
T=[285.00000001]
T=[285.]
T=[284.99998518]
T=[284.99998518]
